In [ ]:
import numpy as np
import pandas as pd 

from nltk.stem.porter import PorterStemmer
import re
import nltk

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from nltk.corpus import stopwords
from sklearn import preprocessing
from keras.models import load_model

choose = input("Choose a model (GRU / LSTM / BiLSTM): ")

model = load_model('/content/drive/MyDrive/final_project/weights_'+ choose +'.h5')
ps = PorterStemmer()
vocab_size = 10000
maxlength = 35
df = pd.read_csv('/content/drive/MyDrive/final_project/Emotion_project.csv') # read the dataset
df = df.dropna() # drop columns with NA values
y = df['Emotion'] # output
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit_transform(y) # encoding the target outputs to integers
nltk.download('stopwords')
stri = input("Input any text: ")
review = re.sub('[^a-zA-Z]', ' ', stri)
review = review.lower()
review = review.split()
review = [ps.stem(word) for word in review if (word == 'not' or word == 'no' or word == 'nor' or not word in stopwords.words('english')) ]
review = ' '.join(review)
onehot_repr = [one_hot(review,vocab_size)] 
embed = pad_sequences(onehot_repr,padding='pre',maxlen=maxlength)
predicti = model.predict(embed)
print(predicti)
print("Label: ",label_encoder.classes_[np.argmax(predicti)])

In [ ]:
import matplotlib.colors as mcolors

!pip3 install ColabTurtle
from ColabTurtle.Turtle import *

def secondmax(list):
  sublist = [x for x in list if x < list.max()]
  indices = np.where(list == max(sublist))
  return max(sublist), indices[0][0]

def transform(list, text) :
    # anger / fear / happy / sadness / surprize
    emotions = [[193, 73, 83], [131, 140, 111], [232, 209, 155], [36, 128, 190], [228, 176, 165]]
    rgb = []
    for i in range(0, 3):
        weight = 0
        for j in range(0, 5):
            alpha = 1
            if list[j] == max(list): alpha = 1.2  # first
            elif list[j] == secondmax(list)[0]: alpha = 1.2  # second
            elif list[j] == min(list): alpha = 0.8  # last
            weight += emotions[j][i] * list[j] * alpha
        weight /= 255
        if weight > 1: weight = 1
        rgb.append(weight)

    (h, s, v) = mcolors.rgb_to_hsv(rgb)      
    light = mcolors.hsv_to_rgb([h, 0.05, 0.9])
    dark = mcolors.hsv_to_rgb([h, s, 0.5])

    percentages = []
    for i in list :
        i *= 1000
        i = int(i)
        i /= 10
        percentages.append(' ' + str(i) + ' %')
    color_dict = {
        list[0]: "Anger",
        list[1]: "Fear",
        list[2]: "Happy",
        list[3]: "Sadness",
        list[4]: "Surprize"
    }
    first = list.argmax()
    second = secondmax(list)[1]

    initializeTurtle()
    penup()
    hideturtle()
    bgcolor("#FFFFFF")
    color(mcolors.to_hex(dark))
    style = (20, 'Verdana', 'italic')
    write(text, align='center', font=style)
    backward(50)
    color(mcolors.to_hex(rgb))
    style = (20, 'Verdana', 'bold')
    write(color_dict[list[first]] + percentages[first] + ' + ' + color_dict[list[second]] + percentages[second], align='center', font=style)

transform(predicti[0],stri)